In [35]:
# Reading in Rasters
# Rasterio tutorial here: 
#https://automating-gis-processes.github.io/CSC18/lessons/L6/reading-raster.html

# First, load packages
import pandas as pd
import os
import fiona
import rasterio
from rasterio.mask import mask
from rasterio.plot import show
import numpy as np
import shapely as sp
import geopandas as gpd
from fiona.crs import from_epsg
import pycrs

os.chdir("/Users/anayahall/projects/grapevine")

from scripts.fxns import epsg_meters

# import matplotlib.pyplot as plt

# from geopandas import GeoSeries, GeoDataFrame
# # only for jupyter nb to show plots inline
%matplotlib inline 


In [15]:
# now load SHAPEFILE for all CA COUNTIES to merge this
county = gpd.read_file("data/raw/CA_Counties/CA_Counties_TIGER2016.shp")


county.head()

# with fiona.open("data/raw/CA_Counties/CA_Counties_TIGER2016.shp", "r") as shapefile:
#     features = [feature["geometry"] for feature in shapefile]

    

,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,06,091,00277310,06091,Sierra,Sierra County,06,H1,G4020,None,None,None,A,2468694587,23299110,+39.5769252,-120.5219926,POLYGON ((-13431319.75097945 4821511.426395644...
1,06,067,00277298,06067,Sacramento,Sacramento County,06,H1,G4020,472,40900,None,A,2499183617,76073827,+38.4500114,-121.3404409,POLYGON ((-13490651.47641084 4680831.603393857...
2,06,083,00277306,06083,Santa Barbara,Santa Barbara County,06,H1,G4020,None,42200,None,A,7084000598,2729814515,+34.5370572,-120.0399729,(POLYGON ((-13423116.77225655 4042044.14856006...
3,06,009,01675885,06009,Calaveras,Calaveras County,06,H1,G4020,None,None,None,A,2641820834,43806026,+38.1838996,-120.5614415,"POLYGON ((-13428575.483353 4627725.227535474, ..."
4,06,111,00277320,06111,Ventura,Ventura County,06,H1,G4020,348,37100,None,A,4773390489,945942791,+34.3587415,-119.1331432,(POLYGON ((-13317853.59433417 3931602.41399276...


In [38]:
lc = "data/interim/CAMLC2010/California Augmented Multisource Landcover 20101.tif"

raster = rasterio.open(lc)
# type(raster)

county = county.to_crs(crs=raster.crs.data)

# # info about raster
print("raster crs: ", raster.crs)
# print("*********")
# print("raster bounds: ", raster.bounds)
# print("*********")
# print("count: ", raster.count)
# print("*********")
# print("driver: ", raster.driver)
# print("*********")
# print("metadata: ", raster.meta)

def getFeatures(gdf):
    """Function to parse features from GeoDataFrame in such a manner that rasterio wants them"""
    import json
    return [json.loads(gdf.to_json())['features'][0]['geometry']]

raster crs:  PROJCS["NAD_1983_California_Albers",GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.2572221010042,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4269"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["standard_parallel_1",34],PARAMETER["standard_parallel_2",40.5],PARAMETER["latitude_of_center",0],PARAMETER["longitude_of_center",-120],PARAMETER["false_easting",0],PARAMETER["false_northing",-4000000],UNIT["metre",1,AUTHORITY["EPSG","9001"]]]


In [19]:
coords = getFeatures(county)
print(coords)

[{'type': 'Polygon', 'coordinates': [[[-56192.57028813594, 186608.41999081755], [-56187.29815550064, 186614.71688614273], [-56177.26862912382, 186639.64760953467], [-56163.76297774525, 186646.6653833855], [-56132.45421291939, 186650.00461850176], [-56118.86191362385, 186669.57745268662], [-56116.834351013684, 186690.11905262945], [-56137.46505207148, 186768.37281738734], [-56143.881897496474, 186832.08392129932], [-56131.09092311979, 186859.7733728839], [-56107.383500501186, 186904.38747642934], [-56116.019546310425, 186932.22491639713], [-56094.3348396558, 187031.51970361127], [-56121.48046053496, 187084.70715642348], [-56172.11535230922, 187273.50141529785], [-56173.458754877975, 187314.7328866301], [-56154.849718410536, 187378.27084885864], [-56130.342648185244, 187414.76812942373], [-56089.56358744841, 187474.3753116331], [-56082.93199129822, 187503.88503177185], [-56098.41465547591, 187557.88068008376], [-56096.48530930342, 187576.6450918913], [-56058.83877828006, 187666.829464206

In [24]:
out_img, out_transform = mask(raster, shapes=coords, crop=True)

In [28]:
out_meta = raster.meta.copy()
print(out_meta)

{'driver': 'GTiff', 'dtype': 'int16', 'nodata': 0.0, 'width': 18280, 'height': 21090, 'count': 1, 'crs': CRS.from_wkt('PROJCS["NAD_1983_California_Albers",GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.2572221010042,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4269"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["standard_parallel_1",34],PARAMETER["standard_parallel_2",40.5],PARAMETER["latitude_of_center",0],PARAMETER["longitude_of_center",-120],PARAMETER["false_easting",0],PARAMETER["false_northing",-4000000],UNIT["metre",1,AUTHORITY["EPSG","9001"]]]'), 'transform': Affine(50.0, 0.0, -373901.4155110012,
       0.0, -50.0, 449991.14220000897)}


In [40]:
# epsg_code = int(raster.crs.data[''][5:])
# print(raster.crs.data)
# print(epsg_code)

KeyError: 'crs'